In [56]:
import glob
import os
import warnings
import mlflow
import mlflow.pytorch 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split
from torchvision import transforms
from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
importlib.reload(help_files._1_visuals_script)
# importlib.reload(help_files._01_load_data)
import  help_files._1_visuals_script  as pauls_vs
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [57]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())
    ### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_run_definitions.py") as file:
    exec(file.read())

In [58]:
# loading data
file_names = ["train_df_3_cat.csv", "test_df_3_cat.csv"]
# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
# Unpack the dataframes into separate variables
train_df, test_df = dataframes

print("DataFrames have been loaded successfully.")


DataFrames have been loaded successfully.


In [59]:
# # end sample or small sample    
if whole_data_set == False:
    print("Using the whole data set")
else:
    train_df = train_df.sample(n=50, random_state=RSEED)
    test_df = test_df.sample(n=50, random_state=RSEED)
    display(Markdown('<span style="color:red"> this is a small sample : 48692</span>'))

<span style="color:red"> this is a small sample : 48692</span>

  definition dataloader (do not change over models)

In [60]:
import pydicom
from torchvision import transforms
from PIL import Image
import torch
from torch.utils.data import Dataset
import numpy as np

class MRILocalizationDataset(Dataset):
    def __init__(self, data, transform=None, has_coordinates=True):
        """
        Args:
            data (pd.DataFrame): DataFrame containing image paths and other info.
            transform (callable, optional): Optional transform to be applied on a sample.
            has_coordinates (bool, optional): Whether the dataset includes x, y coordinates.
        """
        self.data = data
        self.transform = transform
        self.has_coordinates = has_coordinates

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the image path
        img_path = self.data.iloc[idx]['image_path']
        
        # Read the DICOM file
        dicom_image = pydicom.dcmread(img_path)
        
        # Convert the DICOM pixel data to a NumPy array
        image_array = dicom_image.pixel_array
        
        # Convert NumPy array to PIL Image
        image = Image.fromarray(image_array)
        
        # Convert to RGB if necessary (some DICOM files might be grayscale)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Apply transformations (e.g., resizing, normalization)
        if self.transform:
            image = self.transform(image)
        
        # Return image with or without coordinates
        if self.has_coordinates:
            x = torch.tensor(self.data.iloc[idx]['x'], dtype=torch.float32)
            y = torch.tensor(self.data.iloc[idx]['y'], dtype=torch.float32)
            return image, torch.tensor([x, y])
        else:
            return image





training

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
# %pip install keras
# %pip install tensorflow
# from tensorflow.keras.callbacks import EarlyStopping  # Import EarlyStopping
import numpy as np  # Import numpy for setting the random seed

# Define the transformation to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the expected input size of the model
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

# Create the dataset
dataset = MRILocalizationDataset(data=train_df, transform=transform)

# Split the dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 32
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load ResNet-50 and set up for classification
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = train_df['severity'].nunique()

model = models.resnet50(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Define loss functions and optimizer
learining_rate = 0.0001 
criterion_cel = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learining_rate)
num_epochs = 50
# Early stopping parameters
# Lists to store loss values for plotting
train_losses_cel = []
val_losses_cel = []

# Early stopping parameters
diverge_count = 0              # Counter for divergence-based stopping
stop_threshold = 0.05           # Threshold for divergence
max_diverge_count = 5         # Max number of epochs with diverging validation loss

patience_counter = 0           # Counter for plateau-based stopping    
patience = 6                 # Number of epochs for validation loss plateau

 
 

# Calculate number of layers in the model
num_layers = len(list(model.children()))


In [62]:
import torch
import torch.optim as optim
import os
import mlflow
import mlflow.pytorch
import matplotlib.pyplot as plt

# Assuming the model, criterion, and optimizer are already defined
# Assuming `train_loader` and `val_loader` are the DataLoader for training and validation sets

# Define the criterion
criterion = nn.MSELoss()

# Initialize variables to keep track of the best validation loss
best_val_loss = float('inf')
best_model_path = "best_model_weights_epoch_1.pt"


# Adjust the model's output layer to match the target tensor size
model.fc = nn.Linear(model.fc.in_features, 2)  # Output x and y coordinates

# MLflow experiment setup
experiment_name = "MRI_Localization_ResNet50"
mlflow.set_experiment(experiment_name)

# Training loop with MLflow tracking
if mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run():
    mlflow.log_param("learning_rate", learining_rate)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("model_architecture", "ResNet-50 for Localization")
    mlflow.log_param("output_coordinates", "x, y")

    # Set descriptive tags for the model
    mlflow.set_tag("model_description", "ResNet-50 for 3 classification and Sagittal T2/STIR and Sagittal T1 images")

    # Example input tensor with the same shape as the model's expected input
    example_input = torch.randn(1, 3, 224, 224)  # Batch size 1, 3 color channels, 224x224 image


    # Initialize lists to store loss values for plotting
    train_losses = []
    val_losses = []

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        train_loss = 0.0
        
        # Training phase
        for images, coordinates in train_loader:
            images, coordinates = images.to(device), coordinates.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, coordinates)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)  # Average training loss for the epoch
        train_losses.append(train_loss)  # Store train loss for plotting

        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():  # Disable gradient tracking for validation
            for images, coordinates in val_loader:
                images, coordinates = images.to(device), coordinates.to(device)
                outputs = model(images)
                loss = criterion(outputs, coordinates)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)  # Average validation loss for the epoch
        val_losses.append(val_loss)  # Store validation loss for plotting

        # Log losses to MLflow
        mlflow.log_metric("train_loss", train_loss, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

        # Save the model at the current epoch
        model_weight_path = f"epoch_{epoch + 1}.pt"
        torch.save(model.state_dict(), model_weight_path)
        mlflow.log_artifact(model_weight_path)  # Log the model weights artifact
        os.remove(model_weight_path)  # Optionally delete local file after logging

        # Check if validation loss improves and save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0  # Reset patience counter if validation loss improves
            best_model_path = f"best_model_weights_epoch_{epoch + 1}.pt"
            torch.save(model.state_dict(), best_model_path)
            mlflow.log_artifact(best_model_path)  # Log the best model weights
            os.remove(best_model_path)  # Optionally, delete the local file after logging
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch + 1} due to lack of validation loss improvement.")
                break

        # Early stopping based on validation loss divergence (optional)
        if val_loss > best_val_loss * (1 + stop_threshold):
            diverge_count += 1
            if diverge_count >= max_diverge_count:
                print(f"Early stopping at epoch {epoch + 1} due to validation loss divergence.")
                break
        else:
            diverge_count = 0  # Reset diverge count if validation loss doesn't diverge


        # Log final model
    example_input_np = example_input.numpy()
    mlflow.pytorch.log_model(model, "final_model", input_example=example_input_np)
        
    # Ensure train_losses and val_losses have the same length as num_epochs
    while len(train_losses) < num_epochs:
        train_losses.append(None)
    while len(val_losses) < num_epochs:
        val_losses.append(None)

    # Plot and log the loss curves as artifacts
    plt.figure(figsize=(10, 5))
    plt.plot(range(num_epochs), train_losses, label='Train Loss')
    plt.plot(range(num_epochs), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig("loss_curve.png")
    mlflow.log_artifact("loss_curve.png")  # Log the loss curve image


KeyboardInterrupt: 

In [ ]:
sssss

test predicion

In [53]:
test_df = test_df.drop(['severity', 'condition', 'level', 'series_id', 'missing_image'], axis=1)

test_df

,study_id,x,y,image_path
6296,2775207739,157.538462,250.197802,data/train_images_origin/2775207739/3249541180...
1496,664153360,260.194392,306.003738,data/train_images_origin/664153360/1076245514/...
5040,2273432465,234.036660,287.804481,data/train_images_origin/2273432465/114306451/...
8486,3777149998,151.528158,190.928463,data/train_images_origin/3777149998/3550597941...
8959,3967802493,265.649924,385.753425,data/train_images_origin/3967802493/2054070341...
6308,2780118855,189.718464,238.157221,data/train_images_origin/2780118855/2064060968...
563,247968996,256.553304,339.950125,data/train_images_origin/247968996/256361821/1...
2189,957176622,304.949451,394.690110,data/train_images_origin/957176622/1889022706/...
9150,4075603869,206.839695,249.160305,data/train_images_origin/4075603869/2921580289...
8081,3605654232,163.569959,244.938272,data/train_images_origin/3605654232/26446529/3...


In [28]:
print(test_df.to_string(index=False, header=True)) 

  study_id          x          y                                            image_path
   4003253 197.100569 289.457306      data/train_images_origin/4003253/702807833/3.dcm
   4646740 235.980844 360.313610     data/train_images_origin/4646740/3486248476/2.dcm
   4646740 235.980844 360.313610    data/train_images_origin/4646740/3666319702/16.dcm
   4646740 235.980844 360.313610    data/train_images_origin/4646740/3486248476/16.dcm
   4646740 235.980844 360.313610    data/train_images_origin/4646740/3666319702/10.dcm
   4646740 235.980844 360.313610    data/train_images_origin/4646740/3666319702/14.dcm
   4646740 235.980844 360.313610     data/train_images_origin/4646740/3486248476/6.dcm
   4646740 235.980844 360.313610    data/train_images_origin/4646740/3486248476/17.dcm
   4646740 235.980844 360.313610     data/train_images_origin/4646740/3486248476/1.dcm
   7143189 199.493128 272.418121      data/train_images_origin/7143189/132939515/1.dcm
   7143189 199.493128 272.418121     data/t

predicting

In [34]:
test_df = test_df.sample(n=10, random_state=RSEED)
print(test_df)

        study_id           x           y                                         image_path
6296  2775207739  157.538462  250.197802  data/train_images_origin/2775207739/3249541180...
1496   664153360  260.194392  306.003738  data/train_images_origin/664153360/1076245514/...
5040  2273432465  234.036660  287.804481  data/train_images_origin/2273432465/114306451/...
8486  3777149998  151.528158  190.928463  data/train_images_origin/3777149998/3550597941...
8959  3967802493  265.649924  385.753425  data/train_images_origin/3967802493/2054070341...
6308  2780118855  189.718464  238.157221  data/train_images_origin/2780118855/2064060968...
563    247968996  256.553304  339.950125  data/train_images_origin/247968996/256361821/1...
2189   957176622  304.949451  394.690110  data/train_images_origin/957176622/1889022706/...
9150  4075603869  206.839695  249.160305  data/train_images_origin/4075603869/2921580289...
8081  3605654232  163.569959  244.938272  data/train_images_origin/3605654232/26

In [54]:
import torch
from torchvision import transforms, models
import pydicom
import numpy as np
from PIL import Image
import pandas as pd

# Step 1: Define and load the model
loaded_model = models.resnet50()
best_weights_path = "C:/Users/HP1/Desktop/Spiced/capstone-project/mlruns/357139645848513089/a44de32f481a4f6da5488b11e9c1a16f/artifacts/epoch_7.pt"
state_dict = torch.load(best_weights_path)

# Update the model's fully connected layer to match the checkpoint's shape
loaded_model.fc = torch.nn.Linear(loaded_model.fc.in_features, 2)  # Update for 2 output coordinates (x, y)

# Load the state dictionary into the model
loaded_model.load_state_dict(state_dict)

loaded_model.eval()  # Set the model to evaluation mode

# Check if CUDA is available and use GPU if possible, otherwise fall back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Ensure the model is moved to the right device
loaded_model = loaded_model.to(device)

# Image preprocessing function for DICOM files
def preprocess_image(image_path):
    dicom_data = pydicom.dcmread(image_path)  # Read DICOM file
    image = dicom_data.pixel_array  # Extract the pixel array from the DICOM file

    original_size = image.shape  # Store the original size (height, width)
    print(f"Original size: {original_size}")  # Debugging output

    # Convert the image array to a PIL Image
    image = Image.fromarray(image.astype(np.float32))  # Convert to float32

    # Convert to RGB if necessary (some DICOM files might be grayscale)
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Apply the necessary transforms
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match model input size
        transforms.ToTensor(),          # Convert image to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ])
    
    # Rescale the image
    image_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device

    return image_tensor, original_size  # Return original image size along with the tensor

# Function to predict x_new, y_new
def predict_coordinates(image_path, model):
    image_tensor, original_size = preprocess_image(image_path)
    with torch.no_grad():  # No gradient calculation for inference
        output = model(image_tensor)  # Get model output (predicted coordinates)
    
    # Assuming the model output is [x_new, y_new]
    x_new, y_new = output.squeeze().cpu().numpy()  # Convert tensor output to numpy array (move to CPU)

    print(f"Predicted coordinates before rescaling: x_new={x_new}, y_new={y_new}")  # Debugging output

    # Rescale the coordinates to match the original image size
    original_width, original_height = original_size[1], original_size[0]
    print(f"Rescaling with original width={original_width} and height={original_height}")  # Debugging output

    x_new_rescaled = x_new * original_width / 224  # Rescale x to original width
    y_new_rescaled = y_new * original_height / 224  # Rescale y to original height

    print(f"Rescaled coordinates: x_new_rescaled={x_new_rescaled}, y_new_rescaled={y_new_rescaled}")  # Debugging output

    return x_new_rescaled, y_new_rescaled

# Adding predictions to the DataFrame
def add_predictions_to_df(test_df, model):
    x_new_list = []
    y_new_list = []

    for index, row in test_df.iterrows():
        image_path = row['image_path']
        x_new, y_new = predict_coordinates(image_path, model)
        x_new_list.append(x_new)
        y_new_list.append(y_new)

    # Adding new predictions to DataFrame
    test_df['x_new'] = x_new_list
    test_df['y_new'] = y_new_list
    return test_df

# Assuming test_df is already defined
test_df = add_predictions_to_df(test_df, loaded_model)

# Now test_df has the x_new and y_new columns
print(test_df)


C:\Users\HP1\AppData\Local\Temp\ipykernel_1696\2826577863.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(best_weights_path)


Original size: (320, 320)
Predicted coordinates before rescaling: x_new=0.2921183705329895, y_new=0.14594843983650208
Rescaling with original width=320 and height=320
Rescaled coordinates: x_new_rescaled=0.4173119366168976, y_new_rescaled=0.2084977626800537
Original size: (512, 512)
Predicted coordinates before rescaling: x_new=0.4279414117336273, y_new=0.3778077960014343
Rescaling with original width=512 and height=512
Rescaled coordinates: x_new_rescaled=0.978151798248291, y_new_rescaled=0.863560676574707
Original size: (640, 640)
Predicted coordinates before rescaling: x_new=-1.3062772750854492, y_new=-2.0666275024414062
Rescaling with original width=640 and height=640
Rescaled coordinates: x_new_rescaled=-3.7322208881378174, y_new_rescaled=-5.9046502113342285
Original size: (384, 384)
Predicted coordinates before rescaling: x_new=0.5627459287643433, y_new=0.8053154349327087
Rescaling with original width=384 and height=384
Rescaled coordinates: x_new_rescaled=0.9647073149681091, y_n

sss

In [20]:
test_data = test_df

In [ ]:
# calculate prbabilities and probability list
# calculate prbabilities and probability list

# FILEPATH: /c:/Users/HP1/Desktop/Spiced/capstone-project/_5_training_localization_3_cat.ipynb
test_dataset = MRILocalizationDataset(data=test_data, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Inference loop with probability extraction
results = []
probabilities_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)

        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Calculate class probabilities
        _, predicted_classes = torch.max(outputs, 1)
        
        # Append predictions and probabilities
        results.append(predicted_classes.item())
        probabilities_list.append(probabilities.cpu().numpy())



In [ ]:
# Convert the list of probabilities to a numpy array for easier manipulation
probabilities_array = np.vstack(probabilities_list)

# Ensure the length of probabilities_array matches the length of test_df
probabilities_array = probabilities_array[:len(test_df)]

# Add the probabilities to the test_df DataFrame
test_df['Probability_Class_0'] = probabilities_array[:, 0]
test_df['Probability_Class_1'] = probabilities_array[:, 1]
 

# Round the probabilities to 4 decimal places
test_df['Probability_Class_0'] = test_df['Probability_Class_0'].round(4)
test_df['Probability_Class_1'] = test_df['Probability_Class_1'].round(4)
 
# Add the predicted classes to the test_df DataFrame
test_df['Predicted_Class'] = results



# Display the updated DataFrame
 
test_df.head()

In [ ]:
from sklearn.metrics import confusion_matrix
# Group by severity and sum the Probability_Class_1
severity_prob_sum_all_classes = test_df[['Probability_Class_0', 'Probability_Class_1']].mean()

# Print the result
severity_prob_sum_all_classes
# Use severity_prob_sum_all_classes as confusion matrix
conf_matrix_prob_df = severity_prob_sum_all_classes 
# Create a confusion matrix from severity and predicted class
conf_matrix_severity_pred = confusion_matrix(test_df['severity'], test_df['Predicted_Class'])

# Create a DataFrame from the confusion matrix
conf_matrix_severity_pred_df = pd.DataFrame(
    conf_matrix_severity_pred, 
    index=[f"Actual {i}" for i in range(len(conf_matrix_severity_pred))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix_severity_pred[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_severity_pred_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')
# Print the confusion matrix DataFrame
print(conf_matrix_prob_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_prob_df.style.background_gradient(cmap='Blues')
